# Basic RAG Pipeline

In [4]:
%pip install -qU langchain
%pip install -qU langchain-community
%pip install -qU langchain-text-splitters
%pip install -qU langchain_openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Import needed libraries

In [5]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Qdrant
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

### Configuration

In [6]:
data_path = './data/apple.txt'
chunk_size = 500

collection_name = 'apple'
embedding_model = OpenAIEmbeddings() # can be any embedding model

language_model_name = 'gpt-3.5-turbo-0125' # can be any language model

### Load and Split Document

In [7]:
raw_documents = TextLoader(data_path, encoding='utf-8').load()
text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

### Store Chunks in Vector Database

In [8]:
db = Qdrant.from_documents(documents, embedding_model, path="./local_qdrant", collection_name=collection_name)

### Query Vector Database

In [9]:
# query = "In what ways did Martina's sense of aesthetics influence the design of the computer's exterior?"
query = "How did Timmy and Martina manage to make the computer both easy to use and powerful with just old parts?"

docs = db.similarity_search_with_score(query)

only_text_results = [result[0].page_content for result in docs]

for result in docs:
    print(f'Score: {result[1]}')
    print(f'Text:\n{result[0].page_content}\n\n')

Score: 0.9336165187266874
Text:
They started working in their garage, with old parts and plenty of creativity. Timmy took care of the computer's insides – ensuring it was easy to use yet powerful. Martina, with her sense of aesthetics, designed the computer's exterior to make it friendly and inviting.


Score: 0.8862990071488093
Text:
One day, as they sat in the garden under a large apple tree, a apple suddenly fell right onto Timmy's head. "Ouch!" he exclaimed, but then they started to laugh. At that moment, Martina had a brilliant idea. "What if we build our own computer, one that's so simple everyone can use it? One for kids, their parents, and even grandparents?"


Score: 0.8672354987975283
Text:
Once upon a time, in a land far, far away named California, where the sun nearly always shines and dreams are big, there lived two very curious friends named Timmy and Martina. Timmy was a tinkerer. He loved playing with old radios and computers, taking them apart and putting them back tog

### RAG

In [10]:
retriever = db.as_retriever()

template = """Answer the following question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model_name = language_model_name)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke(query)

"Timmy took care of the computer's insides, ensuring it was easy to use yet powerful, while Martina designed the computer's exterior to make it friendly and inviting."